In [203]:
# -*- coding: utf-8 -*-
# Recurrent Neural Network

import copy
import os
import numpy as np
import json
import math
import random
import time

# Parameters Preset
alpha = 0.01
lamda = 0.001
mask = 10

# Data load in
data = []
training_set = []
raw_data = open("user_cart.json",'r')
unum = raw_data.read().count('\n')
raw_data.seek(0)
unum = 0
inum = 0
lines = raw_data.readlines()
for li in lines:
    record = json.loads(li)
    if(len(record)>10):
        thisline = []
        for i in range(len(record)):
            thisline.append(int(record[i]))
            if(int(record[i])>inum):
                inum = int(record[i])
        data.append(thisline)
        training_set.append(thisline[0:int(len(record)*0.8)])
        unum = unum + 1
print unum, inum
# Data understanding
# user_size 703(737) item_size 8533

702 8532


In [204]:
# Represent -- 10D features
np.random.seed(0)
I = np.random.randn(inum+1, mask) * 0.5
H = np.zeros((unum+1, mask))
U = np.random.randn(mask, mask) * 0.5
R = np.random.randn(mask, mask) * 0.5

# The random J choose
opt_set = []
count = 0
for user in xrange(unum):
    new_rand = []
    for item in xrange(len(training_set[user])):
        j = random.randint(0,8532)
        while(j == training_set[user][item]):
            j = random.randint(0, 8532)
        new_rand.append(j)
    opt_set.append(new_rand)
print len(opt_set)

702


In [205]:
def sigmond(x):
    return .5 *(1 + np.tanh(.5 * x))

In [1]:
def train():
    global I, U, R, H
    for user in xrange(unum):
        isum = len(training_set[user])
        
        H3 = np.zeros((1, 10))
        H3 = H3[0]
        # The first item
        item = 0
        #feed-forward
        i = training_set[user][item] -1 
        I2 = I[i]
        A2 = np.dot(H3, R) + np.dot(I2, U)
        H2 = np.zeros((1, mask))
        H2 = H2[0]
        for k in xrange(mask):
            H2[k] = sigmond(A2[k])
        
        # The second item
        item = 1
        #feed-forward
        i = training_set[user][item] -1
        j = opt_set[user][item] -1
        I1 = I[i]
        J1 = I[j]
        A1 = np.dot(H2, R) + np.dot(I1, U)
        H1 = np.zeros((1, 10))
        H1 = H1[0]
        for k in xrange(mask):
            H1[k] = sigmond(A1[k])
        X = np.dot(H2, I1.T) - np.dot(H2, J1.T)
        dI1 = H2
        dJ1 = -H2
        I[i] += alpha *((1 - sigmond(X))* dI1 - lamda * I1)
        I[j] += alpha *((1 - sigmond(X))* dJ1 - lamda * J1)
        I1 = I[i]
        J1 = I[j]
        # back-propagation for once
        dXH2 = I1 - J1
        dHA2 = np.zeros((1, 10))
        dHA2 = dHA2[0]
        for k in xrange(mask):
            dHA2[k] = H2[k] *(1 - H2[k])
        dU = np.zeros((10, 10))
        dR = np.zeros((10, 10))
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        
        #the third item
        item = 2
        #feed-forward
        i = training_set[user][item]-1
        j = opt_set[user][item]-1
        Inow = I[i]
        Jnow = I[j]
        X = np.dot(H1, Inow.T) - np.dot(H1, Jnow.T)
        dXInow = H1
        dXJnow = -H1
        I[i] += alpha *((1 - sigmond(X))* dXInow - lamda * Inow)
        I[j] += alpha *((1 - sigmond(X))* dXJnow - lamda * Jnow)
        Inow = I[i]
        Jnow = I[j]
        #back-propagation --first
        dXH1 = Inow - Jnow
        dHA1 = np.zeros((1, 10))
        for k in xrange(mask):
            dHA1[0][k] = H1[k] *(1 - H1[k])
        dU = np.zeros((10, 10))
        dR = np.zeros((10, 10))
        dU1 = np.zeros((10, 10))
        for k in xrange(mask):
            dU[k] = I1[k] * dHA1 * dXH1
            dR[k] = H2[k] * dHA1 * dXH1 
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        #back-propagation --second
        dXH2 = np.dot(dHA1*dXH1, R.T)
        for k in xrange(mask):
            dU[k] = I2[k] * dHA2 * dXH2
            dR[k] = H3[k] * dHA2 * dXH2
        Ut = U
        Rt = R
        U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
        R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
        
        #    I2   I1 Inow Inext
        #    J2   J1 Jnow Jnext
        #  dHA2 dHA1 
        #  dXH2 dXH1
        #H3  H2   H1 Hnow
        
        # the forth and other items
        item = 3
        Hnow = np.zeros((1, 10))
        Hnow = Hnow[0]
        while item < isum:
            #feed-forward
            i = training_set[user][item]-1
            j = opt_set[user][item]-1
            Inext = I[i]
            Jnext = I[j]
            Anow = np.dot(Inow, U) + np.dot(Jnow, R)
            for k in xrange(mask):
                Hnow[k] = sigmond(Anow[k])
            X = np.dot(Hnow, Inext.T) - np.dot(Hnow, Jnext.T)
            dXInext = Hnow 
            dXJnext = -Hnow
            I[i] += alpha *((1 - sigmond(X))* dXInext - lamda * Inext)
            I[j] += alpha *((1 - sigmond(X))* dXJnext - lamda * Jnext)
            Inext = I[i]
            Jnext = I[j]
            #back-propagation --first
            dXHnow = Inext - Jnext
            dHAnow = np.zeros((1, 10))
            for k in xrange(mask):
                dHAnow[0][k] = Hnow[k] *(1 - Hnow[k])
            dU = np.zeros((10, 10))
            dR = np.zeros((10, 10))
            for k in xrange(mask):
                dU[k] = Inow[k] * dHAnow * dXHnow
                dR[k] = H1[k] * dHAnow * dXHnow
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
            
            #back-propagation --second
            dXH1 = np.dot(dHAnow*dXHnow, R.T)
            for k in xrange(mask):
                dU[k] = I1[k] * dHA1 * dXH1 ###TOHAVEWRONG!!!!!
                dR[k] = H2[k] * dHA1 * dXH1
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)
            
            #back-propagation --third
            dXH2 = np.dot(dHA1*dXH1, R.T)
            for k in xrange(mask):
                dU[k] = I2[k] * dHA2 * dXH2
                dR[k] = H3[k] * dHA2 * dXH2
            Ut = U
            Rt = R
            U += alpha *((1 - sigmond(X))* dU - lamda * Ut)
            R += alpha *((1 - sigmond(X))* dR - lamda * Rt)           
            
            #The next step
            H3 = H2
            H2 = H1
            H1 = Hnow
            #Hnow and Hnext will be calculate at first
            I2 = I1 ; J2 = J1
            I1 = Inow ; J1 = Jnow
            Inow = Inext ; Jnow = Jnext 
            #I/Jnext will be found at first
            dHA2 = dHA1
            dHA1 = dHAnow
            item += 1
        H[user] = Hnow
train()        

NameError: global name 'unum' is not defined

In [ ]:
def predict():
    TP = 0
    P = 0
    user = 0
    while user < unum:
        H0 = np.zeros((1, 10))
        H0[0] = H[user]
        isum = len(data[user])
        if isum > 1:
            item = len(training_set[user])
            while item < isum - 1:
                temp = np.dot(H0, I.T)
                sort_list = np.argsort(-temp[0])
                for k in xrange(mask): #recall 10
                    if (sort_list[k]+1) == data[user][item+1]:
                        TP += 1
                P += 1
                It = I[data[user][item+1]-1]
                At = np.dot(H0,R) + np.dot(It,U)
                Ht = np.zeros((1, 10))
                for k in xrange(mask):
                    Ht[0][k] = sigmond(At[0][k])
                H0 = Ht
                item += 1
        user += 1
    return TP, P

In [212]:
for i in xrange(100):
    print "iter", i, time.ctime()
    train()
    TP , P = predict()
    result = float(TP) / P
    print TP, P, result

iter 0 Wed Oct  5 21:49:41 2016
8639 30117 0.286847959624
iter 1 Wed Oct  5 21:51:42 2016
8644 30117 0.287013978816
iter 2 Wed Oct  5 21:52:53 2016
8649 30117 0.287179998008
iter 3 Wed Oct  5 21:53:52 2016
8645 30117 0.287047182654
iter 4 Wed Oct  5 21:54:55 2016
8644 30117 0.287013978816
iter 5 Wed Oct  5 21:55:56 2016
8641 30117 0.286914367301
iter 6 Wed Oct  5 21:56:56 2016
8637 30117 0.286781551947
iter 7 Wed Oct  5 21:57:56 2016
8638 30117 0.286814755786
iter 8 Wed Oct  5 21:58:55 2016
8636 30117 0.286748348109
iter 9 Wed Oct  5 21:59:55 2016
8637 30117 0.286781551947
iter 10 Wed Oct  5 22:00:55 2016
8636 30117 0.286748348109
iter 11 Wed Oct  5 22:01:55 2016
8638 30117 0.286814755786
iter 12 Wed Oct  5 22:02:55 2016
8654 30117 0.2873460172
iter 13 Wed Oct  5 22:03:55 2016
8645 30117 0.287047182654
iter 14 Wed Oct  5 22:04:55 2016
8644 30117 0.287013978816
iter 15 Wed Oct  5 22:05:55 2016
8645 30117 0.287047182654
iter 16 Wed Oct  5 22:06:55 2016
8644 30117 0.287013978816
iter 17 W

In [ ]:
iter 0 Wed Oct  5 21:49:41 2016
8639 30117 0.286847959624
iter 1 Wed Oct  5 21:51:42 2016
8644 30117 0.287013978816
iter 2 Wed Oct  5 21:52:53 2016
8649 30117 0.287179998008
iter 3 Wed Oct  5 21:53:52 2016
8645 30117 0.287047182654
iter 4 Wed Oct  5 21:54:55 2016
8644 30117 0.287013978816
iter 5 Wed Oct  5 21:55:56 2016
8641 30117 0.286914367301
iter 6 Wed Oct  5 21:56:56 2016
8637 30117 0.286781551947
iter 7 Wed Oct  5 21:57:56 2016
8638 30117 0.286814755786
iter 8 Wed Oct  5 21:58:55 2016
8636 30117 0.286748348109
iter 9 Wed Oct  5 21:59:55 2016
8637 30117 0.286781551947
iter 10 Wed Oct  5 22:00:55 2016
8636 30117 0.286748348109
iter 11 Wed Oct  5 22:01:55 2016
8638 30117 0.286814755786
iter 12 Wed Oct  5 22:02:55 2016
8654 30117 0.2873460172
iter 13 Wed Oct  5 22:03:55 2016
8645 30117 0.287047182654
iter 14 Wed Oct  5 22:04:55 2016
8644 30117 0.287013978816
iter 15 Wed Oct  5 22:05:55 2016
8645 30117 0.287047182654
iter 16 Wed Oct  5 22:06:55 2016
8644 30117 0.287013978816
iter 17 Wed Oct  5 22:08:04 2016
8644 30117 0.287013978816
iter 18 Wed Oct  5 22:09:26 2016
8645 30117 0.287047182654
iter 19 Wed Oct  5 22:11:31 2016
8643 30117 0.286980774978
iter 20 Wed Oct  5 22:13:53 2016
8636 30117 0.286748348109
iter 21 Wed Oct  5 22:16:20 2016
8636 30117 0.286748348109
iter 22 Wed Oct  5 22:18:20 2016
8633 30117 0.286648736594
iter 23 Wed Oct  5 22:19:29 2016
8632 30117 0.286615532756
iter 24 Wed Oct  5 22:21:08 2016
8632 30117 0.286615532756
iter 25 Wed Oct  5 22:24:57 2016
8629 30117 0.28651592124
iter 26 Wed Oct  5 22:28:59 2016
8604 30117 0.285685825281
iter 27 Wed Oct  5 22:32:57 2016
8604 30117 0.285685825281
iter 28 Wed Oct  5 22:36:53 2016
8610 30117 0.285885048312
iter 29 Wed Oct  5 22:41:06 2016
8606 30117 0.285752232958
iter 30 Wed Oct  5 22:45:19 2016
8607 30117 0.285785436796
iter 31 Wed Oct  5 22:49:10 2016
8612 30117 0.285951455988
iter 32 Wed Oct  5 22:52:31 2016
8617 30117 0.28611747518
iter 33 Wed Oct  5 22:55:30 2016
8633 30117 0.286648736594
iter 34 Wed Oct  5 22:58:41 2016
8637 30117 0.286781551947
iter 35 Wed Oct  5 23:01:33 2016
8640 30117 0.286881163462
iter 36 Wed Oct  5 23:04:25 2016
8704 30117 0.289006209118
iter 37 Wed Oct  5 23:07:40 2016
8758 30117 0.290799216389
iter 38 Wed Oct  5 23:10:47 2016
8762 30117 0.290932031743
iter 39 Wed Oct  5 23:14:06 2016
8764 30117 0.29099843942
iter 40 Wed Oct  5 23:17:19 2016
8764 30117 0.29099843942
iter 41 Wed Oct  5 23:20:15 2016
8764 30117 0.29099843942
iter 42 Wed Oct  5 23:23:13 2016
8764 30117 0.29099843942
iter 43 Wed Oct  5 23:26:23 2016
8764 30117 0.29099843942
iter 44 Wed Oct  5 23:29:36 2016
8767 30117 0.291098050935
iter 45 Wed Oct  5 23:32:26 2016
8767 30117 0.291098050935
iter 46 Wed Oct  5 23:35:42 2016
8766 30117 0.291064847096
iter 47 Wed Oct  5 23:39:13 2016
8765 30117 0.291031643258
iter 48 Wed Oct  5 23:41:57 2016
8765 30117 0.291031643258
iter 49 Wed Oct  5 23:45:00 2016
8766 30117 0.291064847096
iter 50 Wed Oct  5 23:47:54 2016
8766 30117 0.291064847096
iter 51 Wed Oct  5 23:51:13 2016
8763 30117 0.290965235581
iter 52 Wed Oct  5 23:54:25 2016
8765 30117 0.291031643258
iter 53 Wed Oct  5 23:57:42 2016
8762 30117 0.290932031743
iter 54 Thu Oct  6 00:01:04 2016
8762 30117 0.290932031743
iter 55 Thu Oct  6 00:04:42 2016
8764 30117 0.29099843942
iter 56 Thu Oct  6 00:08:35 2016
8762 30117 0.290932031743
iter 57 Thu Oct  6 00:12:24 2016
8763 30117 0.290965235581
iter 58 Thu Oct  6 00:16:15 2016
8764 30117 0.29099843942
iter 59 Thu Oct  6 00:20:06 2016
8765 30117 0.291031643258
iter 60 Thu Oct  6 00:23:56 2016
8770 30117 0.29119766245
iter 61 Thu Oct  6 00:27:33 2016
8817 30117 0.292758242853
iter 62 Thu Oct  6 00:31:10 2016
8816 30117 0.292725039015
iter 63 Thu Oct  6 00:33:45 2016
8813 30117 0.292625427499
iter 64 Thu Oct  6 00:36:24 2016
8813 30117 0.292625427499
iter 65 Thu Oct  6 00:39:00 2016
8817 30117 0.292758242853
iter 66 Thu Oct  6 00:41:40 2016
8829 30117 0.293156688913
iter 67 Thu Oct  6 00:44:15 2016
8829 30117 0.293156688913
iter 68 Thu Oct  6 00:46:54 2016
8848 30117 0.293787561842
iter 69 Thu Oct  6 00:49:25 2016
8848 30117 0.293787561842
iter 70 Thu Oct  6 00:51:56 2016
8848 30117 0.293787561842
iter 71 Thu Oct  6 00:54:27 2016
8847 30117 0.293754358004
iter 72 Thu Oct  6 00:56:56 2016
8847 30117 0.293754358004
iter 73 Thu Oct  6 00:59:30 2016
8848 30117 0.293787561842
iter 74 Thu Oct  6 01:02:03 2016
8848 30117 0.293787561842
iter 75 Thu Oct  6 01:04:37 2016
8850 30117 0.293853969519
iter 76 Thu Oct  6 01:07:13 2016
8852 30117 0.293920377196
iter 77 Thu Oct  6 01:09:50 2016
8854 30117 0.293986784872
iter 78 Thu Oct  6 01:12:27 2016
8853 30117 0.293953581034
iter 79 Thu Oct  6 01:15:03 2016
8852 30117 0.293920377196
iter 80 Thu Oct  6 01:17:39 2016
8847 30117 0.293754358004
iter 81 Thu Oct  6 01:20:12 2016
8849 30117 0.293820765681
iter 82 Thu Oct  6 01:22:46 2016
8878 30117 0.294783676993
iter 83 Thu Oct  6 01:25:21 2016
8878 30117 0.294783676993
iter 84 Thu Oct  6 01:27:44 2016
8880 30117 0.29485008467
iter 85 Thu Oct  6 01:29:47 2016
8881 30117 0.294883288508
iter 86 Thu Oct  6 01:31:50 2016
8880 30117 0.29485008467
iter 87 Thu Oct  6 01:33:52 2016
8880 30117 0.29485008467
iter 88 Thu Oct  6 01:35:55 2016
8880 30117 0.29485008467
iter 89 Thu Oct  6 01:37:57 2016
8882 30117 0.294916492347
iter 90 Thu Oct  6 01:39:59 2016
8883 30117 0.294949696185
iter 91 Thu Oct  6 01:42:02 2016
8884 30117 0.294982900023
iter 92 Thu Oct  6 01:44:04 2016
8884 30117 0.294982900023
iter 93 Thu Oct  6 01:46:06 2016
8883 30117 0.294949696185
iter 94 Thu Oct  6 01:48:08 2016
8876 30117 0.294717269316
iter 95 Thu Oct  6 01:50:10 2016
8877 30117 0.294750473155
iter 96 Thu Oct  6 01:52:12 2016
8875 30117 0.294684065478
iter 97 Thu Oct  6 01:54:13 2016
8876 30117 0.294717269316
iter 98 Thu Oct  6 01:56:15 2016
8876 30117 0.294717269316
iter 99 Thu Oct  6 01:58:16 2016
8877 30117 0.294750473155
0


In [ ]:
iter 0 Wed Oct  5 20:20:36 2016
8602 30117 0.285619417605
iter 1 Wed Oct  5 20:21:35 2016
8603 30117 0.285652621443
iter 2 Wed Oct  5 20:22:32 2016
8604 30117 0.285685825281
iter 3 Wed Oct  5 20:23:30 2016
8608 30117 0.285818640635
iter 4 Wed Oct  5 20:24:27 2016
8611 30117 0.28591825215
iter 5 Wed Oct  5 20:25:24 2016
8610 30117 0.285885048312
iter 6 Wed Oct  5 20:26:28 2016
8611 30117 0.28591825215
iter 7 Wed Oct  5 20:27:25 2016
8600 30117 0.285553009928
iter 8 Wed Oct  5 20:28:27 2016
8596 30117 0.285420194574
iter 9 Wed Oct  5 20:29:40 2016
8597 30117 0.285453398413
iter 10 Wed Oct  5 20:30:46 2016
8593 30117 0.285320583059
iter 11 Wed Oct  5 20:31:47 2016
8594 30117 0.285353786898
iter 12 Wed Oct  5 20:32:44 2016
8596 30117 0.285420194574
iter 13 Wed Oct  5 20:33:42 2016
8599 30117 0.28551980609
iter 14 Wed Oct  5 20:34:40 2016
8601 30117 0.285586213766
iter 15 Wed Oct  5 20:35:37 2016
8600 30117 0.285553009928
iter 16 Wed Oct  5 20:36:42 2016
8602 30117 0.285619417605
iter 17 Wed Oct  5 20:37:48 2016
8599 30117 0.28551980609
iter 18 Wed Oct  5 20:39:06 2016
8597 30117 0.285453398413
iter 19 Wed Oct  5 20:40:06 2016
8595 30117 0.285386990736
iter 20 Wed Oct  5 20:41:05 2016
8593 30117 0.285320583059
iter 21 Wed Oct  5 20:42:05 2016
8594 30117 0.285353786898
iter 22 Wed Oct  5 20:43:04 2016
8594 30117 0.285353786898
iter 23 Wed Oct  5 20:44:03 2016
8603 30117 0.285652621443
iter 24 Wed Oct  5 20:45:09 2016
8603 30117 0.285652621443
iter 25 Wed Oct  5 20:46:13 2016
8604 30117 0.285685825281
iter 26 Wed Oct  5 20:47:12 2016
8607 30117 0.285785436796
iter 27 Wed Oct  5 20:48:11 2016
8608 30117 0.285818640635
iter 28 Wed Oct  5 20:49:10 2016
8605 30117 0.28571902912
iter 29 Wed Oct  5 20:50:12 2016
8602 30117 0.285619417605
iter 30 Wed Oct  5 20:51:13 2016
8603 30117 0.285652621443
iter 31 Wed Oct  5 20:52:15 2016
8604 30117 0.285685825281
iter 32 Wed Oct  5 20:53:16 2016
8603 30117 0.285652621443
iter 33 Wed Oct  5 20:54:15 2016
8617 30117 0.28611747518
iter 34 Wed Oct  5 20:55:16 2016
8618 30117 0.286150679018
iter 35 Wed Oct  5 20:56:21 2016
8621 30117 0.286250290534
iter 36 Wed Oct  5 20:57:38 2016
8623 30117 0.28631669821
iter 37 Wed Oct  5 20:58:39 2016
8631 30117 0.286582328917
iter 38 Wed Oct  5 20:59:43 2016
8637 30117 0.286781551947
iter 39 Wed Oct  5 21:00:46 2016
8637 30117 0.286781551947
iter 40 Wed Oct  5 21:01:51 2016
8636 30117 0.286748348109
iter 41 Wed Oct  5 21:02:54 2016
8636 30117 0.286748348109
iter 42 Wed Oct  5 21:03:59 2016
8638 30117 0.286814755786
iter 43 Wed Oct  5 21:05:00 2016
8640 30117 0.286881163462
iter 44 Wed Oct  5 21:06:11 2016
8639 30117 0.286847959624
iter 45 Wed Oct  5 21:08:06 2016
8640 30117 0.286881163462
iter 46 Wed Oct  5 21:10:14 2016
8638 30117 0.286814755786
iter 47 Wed Oct  5 21:11:24 2016
8637 30117 0.286781551947
iter 48 Wed Oct  5 21:12:35 2016
8639 30117 0.286847959624
iter 49 Wed Oct  5 21:14:19 2016
8637 30117 0.286781551947

In [ ]:
iter 0 Wed Oct  5 12:41:32 2016
6968 30117 0.231364345718
iter 1 Wed Oct  5 12:42:28 2016
6975 30117 0.231596772587
iter 2 Wed Oct  5 12:43:26 2016
6978 30117 0.231696384102
iter 3 Wed Oct  5 12:44:23 2016
6985 30117 0.231928810971
iter 4 Wed Oct  5 12:45:19 2016
7006 30117 0.232626091576
iter 5 Wed Oct  5 12:46:16 2016
7008 30117 0.232692499253
iter 6 Wed Oct  5 12:47:13 2016
7027 30117 0.233323372182
iter 7 Wed Oct  5 12:48:09 2016
7048 30117 0.234020652787
iter 8 Wed Oct  5 12:49:05 2016
7051 30117 0.234120264303
iter 9 Wed Oct  5 12:50:02 2016
7058 30117 0.234352691171
iter 10 Wed Oct  5 12:51:02 2016
7060 30117 0.234419098848
iter 11 Wed Oct  5 12:51:59 2016
7061 30117 0.234452302686
iter 12 Wed Oct  5 12:52:56 2016
7066 30117 0.234618321878
iter 13 Wed Oct  5 12:53:54 2016
7076 30117 0.234950360262
iter 14 Wed Oct  5 12:54:51 2016
7093 30117 0.235514825514
iter 15 Wed Oct  5 12:55:48 2016
7099 30117 0.235714048544
iter 16 Wed Oct  5 12:56:46 2016
7104 30117 0.235880067736
iter 17 Wed Oct  5 12:57:43 2016
7118 30117 0.236344921473
iter 18 Wed Oct  5 12:58:40 2016
7115 30117 0.236245309958
iter 19 Wed Oct  5 12:59:36 2016
7119 30117 0.236378125311
iter 20 Wed Oct  5 13:00:33 2016
7164 30117 0.237872298038
iter 21 Wed Oct  5 13:01:30 2016
7167 30117 0.237971909553
iter 22 Wed Oct  5 13:02:27 2016
7167 30117 0.237971909553
iter 23 Wed Oct  5 13:03:23 2016
7178 30117 0.238337151775
iter 24 Wed Oct  5 13:04:29 2016
7184 30117 0.238536374805
iter 25 Wed Oct  5 13:05:30 2016
7184 30117 0.238536374805
iter 26 Wed Oct  5 13:06:27 2016
7187 30117 0.23863598632
iter 27 Wed Oct  5 13:07:23 2016
7186 30117 0.238602782482
iter 28 Wed Oct  5 13:08:19 2016
7219 30117 0.239698509148
iter 29 Wed Oct  5 13:09:16 2016
7224 30117 0.239864528339
iter 30 Wed Oct  5 13:10:14 2016
7228 30117 0.239997343693
iter 31 Wed Oct  5 13:11:13 2016
7246 30117 0.240595012783
iter 32 Wed Oct  5 13:12:11 2016
7267 30117 0.241292293389
iter 33 Wed Oct  5 13:13:08 2016
7271 30117 0.241425108743
iter 34 Wed Oct  5 13:14:05 2016
7276 30117 0.241591127934
iter 35 Wed Oct  5 13:15:03 2016
7323 30117 0.243151708337
iter 36 Wed Oct  5 13:16:00 2016
7328 30117 0.243317727529
iter 37 Wed Oct  5 13:16:57 2016
7351 30117 0.244081415812
iter 38 Wed Oct  5 13:17:55 2016
7354 30117 0.244181027327
iter 39 Wed Oct  5 13:18:52 2016
7361 30117 0.244413454195
iter 40 Wed Oct  5 13:19:49 2016
7380 30117 0.245044327124
iter 41 Wed Oct  5 13:20:47 2016
7386 30117 0.245243550154
iter 42 Wed Oct  5 13:21:44 2016
7396 30117 0.245575588538
iter 43 Wed Oct  5 13:22:41 2016
7399 30117 0.245675200053
iter 44 Wed Oct  5 13:23:39 2016
7398 30117 0.245641996215
iter 45 Wed Oct  5 13:24:36 2016
7402 30117 0.245774811568
iter 46 Wed Oct  5 13:25:33 2016
7407 30117 0.24594083076
iter 47 Wed Oct  5 13:26:31 2016
7433 30117 0.246804130557
iter 48 Wed Oct  5 13:27:28 2016
7434 30117 0.246837334396
iter 49 Wed Oct  5 13:28:25 2016
7438 30117 0.246970149749
iter 50 Wed Oct  5 13:29:22 2016
7461 30117 0.247733838032
iter 51 Wed Oct  5 13:30:20 2016
7495 30117 0.248862768536
iter 52 Wed Oct  5 13:31:17 2016
7529 30117 0.24999169904
iter 53 Wed Oct  5 13:32:14 2016
7533 30117 0.250124514394
iter 54 Wed Oct  5 13:33:11 2016
7545 30117 0.250522960454
iter 55 Wed Oct  5 13:34:08 2016
7553 30117 0.250788591161
iter 56 Wed Oct  5 13:35:05 2016
7558 30117 0.250954610353
iter 57 Wed Oct  5 13:36:02 2016
7568 30117 0.251286648737
iter 58 Wed Oct  5 13:37:00 2016
7598 30117 0.252282763888
iter 59 Wed Oct  5 13:37:57 2016
7604 30117 0.252481986918
iter 60 Wed Oct  5 13:38:54 2016
7606 30117 0.252548394594
iter 61 Wed Oct  5 13:39:52 2016
7632 30117 0.253411694392
iter 62 Wed Oct  5 13:40:49 2016
7639 30117 0.25364412126
iter 63 Wed Oct  5 13:41:46 2016
7657 30117 0.254241790351
iter 64 Wed Oct  5 13:42:44 2016
7668 30117 0.254607032573
iter 65 Wed Oct  5 13:43:41 2016
7668 30117 0.254607032573
iter 66 Wed Oct  5 13:44:38 2016
7671 30117 0.254706644088
iter 67 Wed Oct  5 13:45:36 2016
7673 30117 0.254773051765
iter 68 Wed Oct  5 13:46:33 2016
7700 30117 0.255669555401
iter 69 Wed Oct  5 13:47:30 2016
7710 30117 0.256001593784
iter 70 Wed Oct  5 13:48:27 2016
7714 30117 0.256134409138
iter 71 Wed Oct  5 13:49:25 2016
7720 30117 0.256333632168
iter 72 Wed Oct  5 13:50:22 2016
7722 30117 0.256400039845
iter 73 Wed Oct  5 13:51:21 2016
7721 30117 0.256366836006
iter 74 Wed Oct  5 13:52:18 2016
7727 30117 0.256566059036
iter 75 Wed Oct  5 13:53:16 2016
7732 30117 0.256732078228
iter 76 Wed Oct  5 13:54:13 2016
7736 30117 0.256864893582
iter 77 Wed Oct  5 13:55:11 2016
7735 30117 0.256831689743
iter 78 Wed Oct  5 13:56:08 2016
7737 30117 0.25689809742
iter 79 Wed Oct  5 13:57:06 2016
7740 30117 0.256997708935
iter 80 Wed Oct  5 13:58:03 2016
7741 30117 0.257030912774
iter 81 Wed Oct  5 13:59:01 2016
7767 30117 0.257894212571
iter 82 Wed Oct  5 13:59:58 2016
7774 30117 0.25812663944
iter 83 Wed Oct  5 14:00:56 2016
7788 30117 0.258591493177
iter 84 Wed Oct  5 14:01:53 2016
7788 30117 0.258591493177
iter 85 Wed Oct  5 14:02:51 2016
7794 30117 0.258790716207
iter 86 Wed Oct  5 14:03:48 2016
7795 30117 0.258823920045
iter 87 Wed Oct  5 14:04:45 2016
7795 30117 0.258823920045
iter 88 Wed Oct  5 14:05:43 2016
7798 30117 0.25892353156
iter 89 Wed Oct  5 14:06:40 2016
7801 30117 0.259023143075
iter 90 Wed Oct  5 14:07:37 2016
7819 30117 0.259620812166
iter 91 Wed Oct  5 14:08:34 2016
7821 30117 0.259687219843
iter 92 Wed Oct  5 14:09:32 2016
7822 30117 0.259720423681
iter 93 Wed Oct  5 14:10:29 2016
7817 30117 0.259554404489
iter 94 Wed Oct  5 14:11:27 2016
7817 30117 0.259554404489
iter 95 Wed Oct  5 14:12:24 2016
7821 30117 0.259687219843
iter 96 Wed Oct  5 14:13:22 2016
7823 30117 0.259753627519
iter 97 Wed Oct  5 14:14:19 2016
7822 30117 0.259720423681
iter 98 Wed Oct  5 14:15:16 2016
7820 30117 0.259654016004
iter 99 Wed Oct  5 14:16:14 2016
7827 30117 0.259886442873
iter 100 Wed Oct  5 14:17:12 2016
7829 30117 0.25995285055
iter 101 Wed Oct  5 14:18:12 2016
7839 30117 0.260284888933
iter 102 Wed Oct  5 14:19:09 2016
7841 30117 0.26035129661
iter 103 Wed Oct  5 14:20:07 2016
7864 30117 0.261114984892
iter 104 Wed Oct  5 14:21:04 2016
7866 30117 0.261181392569
iter 105 Wed Oct  5 14:22:02 2016
7869 30117 0.261281004084
iter 106 Wed Oct  5 14:22:59 2016
7868 30117 0.261247800246
iter 107 Wed Oct  5 14:23:56 2016
7874 30117 0.261447023276
iter 108 Wed Oct  5 14:24:54 2016
7878 30117 0.261579838629
iter 109 Wed Oct  5 14:25:52 2016
7878 30117 0.261579838629
iter 110 Wed Oct  5 14:26:49 2016
7881 30117 0.261679450144
iter 111 Wed Oct  5 14:27:47 2016
7893 30117 0.262077896205
iter 112 Wed Oct  5 14:28:44 2016
7900 30117 0.262310323073
iter 113 Wed Oct  5 14:29:41 2016
7919 30117 0.262941196002
iter 114 Wed Oct  5 14:30:39 2016
7913 30117 0.262741972972
iter 115 Wed Oct  5 14:31:36 2016
7920 30117 0.262974399841
iter 116 Wed Oct  5 14:32:33 2016
7926 30117 0.263173622871
iter 117 Wed Oct  5 14:33:31 2016
7922 30117 0.263040807517
iter 118 Wed Oct  5 14:34:28 2016
7937 30117 0.263538865093
iter 119 Wed Oct  5 14:35:25 2016
7938 30117 0.263572068931
iter 120 Wed Oct  5 14:36:23 2016
7941 30117 0.263671680446
iter 121 Wed Oct  5 14:37:20 2016
7944 30117 0.263771291961
iter 122 Wed Oct  5 14:38:17 2016
7946 30117 0.263837699638
iter 123 Wed Oct  5 14:39:15 2016
7947 30117 0.263870903476
iter 124 Wed Oct  5 14:40:12 2016
7947 30117 0.263870903476
iter 125 Wed Oct  5 14:41:10 2016
7946 30117 0.263837699638
iter 126 Wed Oct  5 14:42:08 2016
7945 30117 0.2638044958
iter 127 Wed Oct  5 14:43:05 2016
7941 30117 0.263671680446
iter 128 Wed Oct  5 14:44:02 2016
7945 30117 0.2638044958
iter 129 Wed Oct  5 14:45:00 2016
7967 30117 0.264534980244
iter 130 Wed Oct  5 14:45:57 2016
7969 30117 0.26460138792
iter 131 Wed Oct  5 14:46:55 2016
8003 30117 0.265730318425
iter 132 Wed Oct  5 14:47:52 2016
8016 30117 0.266161968324
iter 133 Wed Oct  5 14:48:49 2016
8021 30117 0.266327987515
iter 134 Wed Oct  5 14:49:47 2016
8026 30117 0.266494006707
iter 135 Wed Oct  5 14:50:44 2016
8026 30117 0.266494006707
iter 136 Wed Oct  5 14:51:42 2016
8036 30117 0.266826045091
iter 137 Wed Oct  5 14:52:39 2016
8048 30117 0.267224491151
iter 138 Wed Oct  5 14:53:37 2016
8051 30117 0.267324102666
iter 139 Wed Oct  5 14:54:34 2016
8059 30117 0.267589733373
iter 140 Wed Oct  5 14:55:31 2016
8057 30117 0.267523325696
iter 141 Wed Oct  5 14:56:29 2016
8057 30117 0.267523325696
iter 142 Wed Oct  5 14:57:27 2016
8070 30117 0.267954975595
iter 143 Wed Oct  5 14:58:24 2016
8079 30117 0.26825381014
iter 144 Wed Oct  5 14:59:22 2016
8084 30117 0.268419829332
iter 145 Wed Oct  5 15:00:19 2016
8092 30117 0.268685460039
iter 146 Wed Oct  5 15:01:17 2016
8093 30117 0.268718663878
iter 147 Wed Oct  5 15:02:14 2016
8102 30117 0.269017498423
iter 148 Wed Oct  5 15:03:12 2016
8104 30117 0.2690839061
iter 149 Wed Oct  5 15:04:09 2016
8109 30117 0.269249925291
iter 150 Wed Oct  5 15:05:06 2016
8118 30117 0.269548759837
iter 151 Wed Oct  5 15:06:04 2016
8122 30117 0.26968157519
iter 152 Wed Oct  5 15:07:01 2016
8120 30117 0.269615167513
iter 153 Wed Oct  5 15:07:59 2016
8136 30117 0.270146428927
iter 154 Wed Oct  5 15:08:56 2016
8142 30117 0.270345651957
iter 155 Wed Oct  5 15:09:54 2016
8172 30117 0.271341767108
iter 156 Wed Oct  5 15:10:51 2016
8177 30117 0.2715077863
iter 157 Wed Oct  5 15:11:49 2016
8175 30117 0.271441378623
iter 158 Wed Oct  5 15:12:46 2016
8184 30117 0.271740213169
iter 159 Wed Oct  5 15:13:44 2016
8189 30117 0.27190623236
iter 160 Wed Oct  5 15:14:41 2016
8191 30117 0.271972640037
iter 161 Wed Oct  5 15:15:39 2016
8192 30117 0.272005843876
iter 162 Wed Oct  5 15:16:37 2016
8196 30117 0.272138659229
iter 163 Wed Oct  5 15:17:34 2016
8203 30117 0.272371086098
iter 164 Wed Oct  5 15:18:31 2016
8206 30117 0.272470697613
iter 165 Wed Oct  5 15:19:29 2016
8205 30117 0.272437493774
iter 166 Wed Oct  5 15:20:26 2016
8226 30117 0.27313477438
iter 167 Wed Oct  5 15:21:24 2016
8229 30117 0.273234385895
iter 168 Wed Oct  5 15:22:21 2016
8233 30117 0.273367201248
iter 169 Wed Oct  5 15:23:18 2016
8233 30117 0.273367201248
iter 170 Wed Oct  5 15:24:16 2016
8236 30117 0.273466812764
iter 171 Wed Oct  5 15:25:13 2016
8241 30117 0.273632831955
iter 172 Wed Oct  5 15:26:11 2016
8248 30117 0.273865258824
iter 173 Wed Oct  5 15:27:08 2016
8278 30117 0.274861373975
iter 174 Wed Oct  5 15:28:06 2016
8280 30117 0.274927781652
iter 175 Wed Oct  5 15:29:03 2016
8282 30117 0.274994189328
iter 176 Wed Oct  5 15:30:01 2016
8290 30117 0.275259820035
iter 177 Wed Oct  5 15:30:58 2016
8310 30117 0.275923896802
iter 178 Wed Oct  5 15:31:56 2016
8311 30117 0.275957100641
iter 179 Wed Oct  5 15:32:54 2016
8313 30117 0.276023508318
iter 180 Wed Oct  5 15:33:51 2016
8312 30117 0.275990304479
iter 181 Wed Oct  5 15:34:49 2016
8358 30117 0.277517681044
iter 182 Wed Oct  5 15:35:46 2016
8369 30117 0.277882923266
iter 183 Wed Oct  5 15:36:43 2016
8376 30117 0.278115350134
iter 184 Wed Oct  5 15:37:41 2016
8386 30117 0.278447388518
iter 185 Wed Oct  5 15:38:38 2016
8391 30117 0.27861340771
iter 186 Wed Oct  5 15:39:36 2016
8385 30117 0.27841418468
iter 187 Wed Oct  5 15:40:33 2016
8388 30117 0.278513796195
iter 188 Wed Oct  5 15:41:30 2016
8402 30117 0.278978649932
iter 189 Wed Oct  5 15:42:28 2016
8408 30117 0.279177872962
iter 190 Wed Oct  5 15:43:25 2016
8410 30117 0.279244280639
iter 191 Wed Oct  5 15:44:23 2016
8418 30117 0.279509911346
iter 192 Wed Oct  5 15:45:20 2016
8422 30117 0.279642726699
iter 193 Wed Oct  5 15:46:18 2016
8429 30117 0.279875153568
iter 194 Wed Oct  5 15:47:15 2016
8431 30117 0.279941561244
iter 195 Wed Oct  5 15:48:12 2016
8415 30117 0.279410299831
iter 196 Wed Oct  5 15:49:10 2016
8415 30117 0.279410299831
iter 197 Wed Oct  5 15:50:07 2016
8450 30117 0.280572434173
iter 198 Wed Oct  5 15:51:05 2016
8519 30117 0.28286349902
iter 199 Wed Oct  5 15:52:02 2016
8520 30117 0.282896702859
iter 200 Wed Oct  5 15:53:00 2016
8524 30117 0.283029518212
iter 201 Wed Oct  5 15:53:57 2016
8526 30117 0.283095925889
iter 202 Wed Oct  5 15:54:54 2016
8528 30117 0.283162333566
iter 203 Wed Oct  5 15:55:52 2016
8535 30117 0.283394760434
iter 204 Wed Oct  5 15:56:49 2016
8542 30117 0.283627187303
iter 205 Wed Oct  5 15:57:46 2016
8547 30117 0.283793206495
iter 206 Wed Oct  5 15:58:43 2016
8550 30117 0.28389281801
iter 207 Wed Oct  5 15:59:41 2016
8551 30117 0.283926021848
iter 208 Wed Oct  5 16:00:38 2016
8552 30117 0.283959225686
iter 209 Wed Oct  5 16:01:35 2016
8555 30117 0.284058837202
iter 210 Wed Oct  5 16:02:33 2016
8556 30117 0.28409204104
iter 211 Wed Oct  5 16:03:30 2016
8553 30117 0.283992429525
iter 212 Wed Oct  5 16:04:28 2016
8550 30117 0.28389281801
iter 213 Wed Oct  5 16:05:25 2016
8579 30117 0.284855729322
iter 214 Wed Oct  5 16:06:23 2016
8586 30117 0.285088156191
iter 215 Wed Oct  5 16:07:20 2016
8583 30117 0.284988544676
iter 216 Wed Oct  5 16:08:18 2016
8586 30117 0.285088156191
iter 217 Wed Oct  5 16:09:15 2016
8581 30117 0.284922136999
iter 218 Wed Oct  5 16:10:12 2016
8582 30117 0.284955340837
iter 219 Wed Oct  5 16:11:10 2016
8586 30117 0.285088156191
iter 220 Wed Oct  5 16:12:07 2016
8589 30117 0.285187767706
iter 221 Wed Oct  5 16:13:05 2016
8588 30117 0.285154563868
iter 222 Wed Oct  5 16:14:02 2016
8589 30117 0.285187767706
iter 223 Wed Oct  5 16:15:00 2016
8590 30117 0.285220971544
iter 224 Wed Oct  5 16:15:57 2016
8589 30117 0.285187767706
iter 225 Wed Oct  5 16:16:55 2016
8590 30117 0.285220971544
iter 226 Wed Oct  5 16:17:52 2016
8592 30117 0.285287379221
iter 227 Wed Oct  5 16:18:50 2016
8596 30117 0.285420194574
iter 228 Wed Oct  5 16:19:47 2016
8592 30117 0.285287379221
iter 229 Wed Oct  5 16:20:44 2016
8594 30117 0.285353786898
iter 230 Wed Oct  5 16:21:42 2016
8594 30117 0.285353786898
iter 231 Wed Oct  5 16:22:40 2016
8598 30117 0.285486602251
iter 232 Wed Oct  5 16:23:37 2016
8601 30117 0.285586213766
iter 233 Wed Oct  5 16:24:34 2016
8647 30117 0.287113590331
iter 234 Wed Oct  5 16:25:32 2016
8647 30117 0.287113590331
iter 235 Wed Oct  5 16:26:30 2016
8647 30117 0.287113590331
iter 236 Wed Oct  5 16:27:27 2016
8648 30117 0.287146794169
iter 237 Wed Oct  5 16:28:25 2016
8652 30117 0.287279609523
iter 238 Wed Oct  5 16:29:22 2016
8650 30117 0.287213201846
iter 239 Wed Oct  5 16:30:20 2016
8651 30117 0.287246405684
iter 240 Wed Oct  5 16:31:17 2016
8645 30117 0.287047182654
iter 241 Wed Oct  5 16:32:15 2016
8640 30117 0.286881163462
iter 242 Wed Oct  5 16:33:12 2016
8571 30117 0.284590098615
iter 243 Wed Oct  5 16:34:10 2016
8571 30117 0.284590098615
iter 244 Wed Oct  5 16:35:07 2016
8569 30117 0.284523690939
iter 245 Wed Oct  5 16:36:05 2016
8566 30117 0.284424079424
iter 246 Wed Oct  5 16:37:03 2016
8566 30117 0.284424079424
iter 247 Wed Oct  5 16:38:00 2016
8571 30117 0.284590098615
iter 248 Wed Oct  5 16:38:58 2016
8575 30117 0.284722913969
iter 249 Wed Oct  5 16:39:56 2016
8574 30117 0.28468971013
iter 250 Wed Oct  5 16:40:53 2016
8583 30117 0.284988544676
iter 251 Wed Oct  5 16:41:50 2016
8586 30117 0.285088156191
iter 252 Wed Oct  5 16:42:48 2016
8584 30117 0.285021748514
iter 253 Wed Oct  5 16:43:45 2016
8585 30117 0.285054952352
iter 254 Wed Oct  5 16:44:43 2016
8581 30117 0.284922136999
iter 255 Wed Oct  5 16:45:41 2016
8580 30117 0.284888933161
iter 256 Wed Oct  5 16:46:38 2016
8581 30117 0.284922136999
iter 257 Wed Oct  5 16:47:36 2016
8580 30117 0.284888933161
iter 258 Wed Oct  5 16:48:34 2016
8580 30117 0.284888933161
iter 259 Wed Oct  5 16:49:31 2016
8582 30117 0.284955340837
iter 260 Wed Oct  5 16:50:29 2016
8580 30117 0.284888933161
iter 261 Wed Oct  5 16:51:27 2016
8578 30117 0.284822525484
iter 262 Wed Oct  5 16:52:24 2016
8593 30117 0.285320583059
iter 263 Wed Oct  5 16:53:22 2016
8595 30117 0.285386990736
iter 264 Wed Oct  5 16:54:20 2016
8595 30117 0.285386990736
iter 265 Wed Oct  5 16:55:17 2016
8597 30117 0.285453398413
iter 266 Wed Oct  5 16:56:15 2016
8598 30117 0.285486602251
iter 267 Wed Oct  5 16:57:12 2016
8600 30117 0.285553009928
iter 268 Wed Oct  5 16:58:09 2016
8601 30117 0.285586213766
iter 269 Wed Oct  5 16:59:07 2016
8603 30117 0.285652621443
0


In [208]:
iter 0 Wed Oct  5 10:20:22 2016
5414 30117 0.179765580901
iter 1 Wed Oct  5 10:22:33 2016
4875 30117 0.161868712023
iter 2 Wed Oct  5 10:23:38 2016
4070 30117 0.13513962214
iter 3 Wed Oct  5 10:24:40 2016
3337 30117 0.11080120862
iter 4 Wed Oct  5 10:25:52 2016
3197 30117 0.106152671249
iter 5 Wed Oct  5 10:26:47 2016
3197 30117 0.106152671249
iter 6 Wed Oct  5 10:27:43 2016
2555 30117 0.0848358070193
iter 7 Wed Oct  5 10:28:39 2016
2543 30117 0.0844373609589
iter 8 Wed Oct  5 10:29:37 2016
2541 30117 0.0843709532822
iter 9 Wed Oct  5 10:30:40 2016
2547 30117 0.0845701763124
iter 10 Wed Oct  5 10:31:38 2016
2784 30117 0.0924394860046
iter 11 Wed Oct  5 10:32:35 2016
2782 30117 0.0923730783279
iter 12 Wed Oct  5 10:33:32 2016
2784 30117 0.0924394860046
iter 13 Wed Oct  5 10:34:28 2016
2780 30117 0.0923066706511
iter 14 Wed Oct  5 10:35:30 2016
2783 30117 0.0924062821662
iter 15 Wed Oct  5 10:36:26 2016
2786 30117 0.0925058936813
iter 16 Wed Oct  5 10:37:22 2016
2790 30117 0.0926387090348
iter 17 Wed Oct  5 10:38:18 2016
2795 30117 0.0928047282266
iter 18 Wed Oct  5 10:39:14 2016
2899 30117 0.0962579274164
iter 19 Wed Oct  5 10:40:10 2016
2900 30117 0.0962911312548
iter 20 Wed Oct  5 10:41:06 2016
2904 30117 0.0964239466082
iter 21 Wed Oct  5 10:42:02 2016
2904 30117 0.0964239466082
iter 22 Wed Oct  5 10:42:59 2016
2903 30117 0.0963907427699
iter 23 Wed Oct  5 10:43:55 2016
2919 30117 0.0969220041837
iter 24 Wed Oct  5 10:44:51 2016
3060 30117 0.101603745393
iter 25 Wed Oct  5 10:45:47 2016
3068 30117 0.1018693761
iter 26 Wed Oct  5 10:46:43 2016
3070 30117 0.101935783777
iter 27 Wed Oct  5 10:47:40 2016
3095 30117 0.102765879736
iter 28 Wed Oct  5 10:48:36 2016
3391 30117 0.112594215891
iter 29 Wed Oct  5 10:49:32 2016
3508 30117 0.11647906498
iter 30 Wed Oct  5 10:50:28 2016
3517 30117 0.116777899525

SyntaxError: invalid syntax (<ipython-input-208-6a5494172606>, line 1)

In [ ]:
iter 0 Wed Oct  5 10:52:25 2016
3526 30117 0.11707673407
iter 1 Wed Oct  5 10:53:22 2016
3518 30117 0.116811103364
iter 2 Wed Oct  5 10:54:18 2016
3523 30117 0.116977122555
iter 3 Wed Oct  5 10:55:15 2016
3527 30117 0.117109937909
iter 4 Wed Oct  5 10:56:12 2016
3541 30117 0.117574791646
iter 5 Wed Oct  5 10:57:08 2016
3822 30117 0.126905070226
iter 6 Wed Oct  5 10:58:05 2016
3918 30117 0.130092638709
iter 7 Wed Oct  5 10:59:01 2016
3930 30117 0.130491084769
iter 8 Wed Oct  5 11:00:01 2016
3932 30117 0.130557492446
iter 9 Wed Oct  5 11:00:58 2016
3935 30117 0.130657103961
iter 10 Wed Oct  5 11:01:55 2016
3938 30117 0.130756715476
iter 11 Wed Oct  5 11:02:52 2016
3940 30117 0.130823123153
iter 12 Wed Oct  5 11:03:49 2016
3942 30117 0.13088953083
iter 13 Wed Oct  5 11:04:47 2016
3955 30117 0.131321180728
iter 14 Wed Oct  5 11:05:48 2016
4041 30117 0.134176710828
iter 15 Wed Oct  5 11:06:45 2016
4175 30117 0.138626025169
iter 16 Wed Oct  5 11:07:43 2016
4232 30117 0.140518643955
iter 17 Wed Oct  5 11:08:40 2016
4255 30117 0.141282332238
iter 18 Wed Oct  5 11:09:38 2016
4499 30117 0.149384068798
iter 19 Wed Oct  5 11:10:35 2016
4628 30117 0.153667363947
iter 20 Wed Oct  5 11:11:32 2016
4761 30117 0.15808347445
iter 21 Wed Oct  5 11:12:29 2016
4796 30117 0.159245608792
iter 22 Wed Oct  5 11:13:26 2016
4809 30117 0.159677258691
iter 23 Wed Oct  5 11:14:23 2016
4815 30117 0.159876481721
iter 24 Wed Oct  5 11:15:21 2016
4822 30117 0.16010890859
iter 25 Wed Oct  5 11:16:18 2016
4979 30117 0.165321911213
iter 26 Wed Oct  5 11:17:16 2016
4982 30117 0.165421522728
iter 27 Wed Oct  5 11:18:21 2016
5031 30117 0.167048510808
iter 28 Wed Oct  5 11:19:24 2016
5099 30117 0.169306371817
iter 29 Wed Oct  5 11:20:30 2016
5109 30117 0.1696384102
iter 30 Wed Oct  5 11:21:38 2016
5169 30117 0.171630640502
iter 31 Wed Oct  5 11:22:36 2016
5191 30117 0.172361124946
iter 32 Wed Oct  5 11:23:34 2016
5243 30117 0.174087724541
iter 33 Wed Oct  5 11:24:30 2016
5252 30117 0.174386559086
iter 34 Wed Oct  5 11:25:26 2016
5283 30117 0.175415878076
iter 35 Wed Oct  5 11:26:23 2016
5305 30117 0.17614636252
iter 36 Wed Oct  5 11:27:21 2016
5379 30117 0.178603446558
iter 37 Wed Oct  5 11:28:21 2016
5389 30117 0.178935484942
iter 38 Wed Oct  5 11:29:18 2016
5407 30117 0.179533154033
iter 39 Wed Oct  5 11:30:14 2016
5446 30117 0.180828103729
iter 40 Wed Oct  5 11:31:10 2016
5498 30117 0.182554703324
iter 41 Wed Oct  5 11:32:07 2016
5527 30117 0.183517614636
iter 42 Wed Oct  5 11:33:03 2016
5550 30117 0.184281302919
iter 43 Wed Oct  5 11:34:00 2016
5563 30117 0.184712952817
iter 44 Wed Oct  5 11:34:57 2016
5589 30117 0.185576252615
iter 45 Wed Oct  5 11:35:56 2016
5652 30117 0.187668094432
iter 46 Wed Oct  5 11:36:53 2016
5673 30117 0.188365375037
iter 47 Wed Oct  5 11:37:54 2016
5712 30117 0.189660324734
iter 48 Wed Oct  5 11:38:52 2016
5719 30117 0.189892751602
iter 49 Wed Oct  5 11:39:50 2016
5726 30117 0.190125178471
iter 50 Wed Oct  5 11:40:48 2016
5733 30117 0.190357605339
iter 51 Wed Oct  5 11:41:46 2016
5740 30117 0.190590032208
iter 52 Wed Oct  5 11:42:43 2016
5756 30117 0.191121293622
iter 53 Wed Oct  5 11:43:42 2016
5759 30117 0.191220905137
iter 54 Wed Oct  5 11:44:38 2016
5777 30117 0.191818574227
iter 55 Wed Oct  5 11:45:35 2016
5812 30117 0.19298070857
iter 56 Wed Oct  5 11:46:31 2016
5816 30117 0.193113523923
iter 57 Wed Oct  5 11:47:28 2016
5822 30117 0.193312746954
iter 58 Wed Oct  5 11:48:24 2016
5849 30117 0.194209250589
iter 59 Wed Oct  5 11:49:25 2016
5860 30117 0.194574492811
iter 60 Wed Oct  5 11:50:22 2016
5875 30117 0.195072550387
iter 61 Wed Oct  5 11:51:18 2016
5874 30117 0.195039346548
iter 62 Wed Oct  5 11:52:15 2016
5887 30117 0.195470996447
iter 63 Wed Oct  5 11:53:11 2016
5922 30117 0.19663313079
iter 64 Wed Oct  5 11:54:07 2016
5924 30117 0.196699538467
iter 65 Wed Oct  5 11:55:06 2016
5925 30117 0.196732742305
iter 66 Wed Oct  5 11:56:03 2016
5924 30117 0.196699538467
iter 67 Wed Oct  5 11:56:59 2016
5925 30117 0.196732742305
iter 68 Wed Oct  5 11:57:56 2016
5929 30117 0.196865557658
iter 69 Wed Oct  5 11:58:53 2016
5931 30117 0.196931965335
iter 70 Wed Oct  5 11:59:54 2016
5940 30117 0.19723079988
iter 71 Wed Oct  5 12:00:50 2016
5957 30117 0.197795265133
iter 72 Wed Oct  5 12:01:47 2016
5960 30117 0.197894876648
iter 73 Wed Oct  5 12:02:44 2016
5962 30117 0.197961284324
iter 74 Wed Oct  5 12:03:40 2016
5968 30117 0.198160507355
iter 75 Wed Oct  5 12:04:37 2016
6006 30117 0.199422253212
iter 76 Wed Oct  5 12:05:33 2016
6072 30117 0.201613706544
iter 77 Wed Oct  5 12:06:29 2016
6135 30117 0.203705548361
iter 78 Wed Oct  5 12:07:26 2016
6158 30117 0.204469236644
iter 79 Wed Oct  5 12:08:22 2016
6205 30117 0.206029817047
iter 80 Wed Oct  5 12:09:19 2016
6223 30117 0.206627486137
iter 81 Wed Oct  5 12:10:15 2016
6264 30117 0.20798884351
iter 82 Wed Oct  5 12:11:11 2016
6270 30117 0.20818806654
iter 83 Wed Oct  5 12:12:08 2016
6273 30117 0.208287678056
iter 84 Wed Oct  5 12:13:04 2016
6293 30117 0.208951754823
iter 85 Wed Oct  5 12:14:01 2016
6294 30117 0.208984958661
iter 86 Wed Oct  5 12:14:57 2016
6326 30117 0.210047481489
iter 87 Wed Oct  5 12:15:54 2016
6329 30117 0.210147093004
iter 88 Wed Oct  5 12:16:50 2016
6338 30117 0.210445927549
iter 89 Wed Oct  5 12:17:46 2016
6348 30117 0.210777965933
iter 90 Wed Oct  5 12:18:43 2016
6406 30117 0.212703788558
iter 91 Wed Oct  5 12:19:39 2016
6432 30117 0.213567088355
iter 92 Wed Oct  5 12:20:35 2016
6534 30117 0.216953879869
iter 93 Wed Oct  5 12:21:32 2016
6611 30117 0.219510575423
iter 94 Wed Oct  5 12:22:28 2016
6619 30117 0.219776206129
iter 95 Wed Oct  5 12:23:25 2016
6740 30117 0.223793870571
iter 96 Wed Oct  5 12:24:21 2016
6761 30117 0.224491151177
iter 97 Wed Oct  5 12:25:17 2016
6772 30117 0.224856393399
iter 98 Wed Oct  5 12:26:14 2016
6785 30117 0.225288043298
iter 99 Wed Oct  5 12:27:10 2016
6797 30117 0.225686489358
iter 100 Wed Oct  5 12:28:06 2016
6826 30117 0.226649400671
iter 101 Wed Oct  5 12:29:03 2016
6834 30117 0.226915031378
iter 102 Wed Oct  5 12:29:59 2016
6845 30117 0.2272802736
iter 103 Wed Oct  5 12:30:55 2016
6861 30117 0.227811535013
iter 104 Wed Oct  5 12:31:52 2016
6866 30117 0.227977554205
iter 105 Wed Oct  5 12:32:49 2016
6877 30117 0.228342796427
iter 106 Wed Oct  5 12:33:45 2016
6908 30117 0.229372115417
iter 107 Wed Oct  5 12:34:42 2016
6916 30117 0.229637746123
iter 108 Wed Oct  5 12:35:38 2016
6928 30117 0.230036192184
iter 109 Wed Oct  5 12:36:35 2016
6937 30117 0.230335026729
iter 110 Wed Oct  5 12:37:31 2016
6943 30117 0.230534249759
iter 111 Wed Oct  5 12:38:27 2016
6948 30117 0.230700268951
iter 112 Wed Oct  5 12:39:24 2016
6953 30117 0.230866288143